In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
data_dir = 'kanker/Training/'
# Tentukan ukuran gambar yang digunakan
image_size = (128, 128)  # Ukuran gambar yang diresize
batch_size = 32  # Ukuran batch untuk training

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)

# Membaca gambar dan membagi data menjadi training
train_generator = train_datagen.flow_from_directory(
    os.path.join(data_dir),
    target_size=image_size,  # Mengubah ukuran gambar
    batch_size=batch_size,
    class_mode='categorical', 
    shuffle=True 
)

# Menampilkan informasi tentang dataset
print(train_generator.class_indices)

Found 5712 images belonging to 4 classes.
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


# Training

In [4]:
# Membangun model CNN
model = tf.keras.Sequential([
    # Layer Konvolusi 1
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Layer Konvolusi 2
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Layer Konvolusi 3
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Flatten hasil konvolusi
    tf.keras.layers.Flatten(),

    # Fully Connected Layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Regularisasi dengan Dropout

    # Output Layer (3 kelas: glioma, nontumor, meningioma)
    tf.keras.layers.Dense(4, activation='softmax')  # Softmax untuk multi-class classification
])

# Kompilasi model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Karena klasifikasi multi-kelas
    metrics=['accuracy']
)

# Ringkasan Model
model.summary()


C:\Users\Acer\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,156 (12.61 MB)

 Trainable params: 3,305,156 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Melatih model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50  # Bisa disesuaikan
)

C:\Users\Acer\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 41s 217ms/step - accuracy: 0.5774 - loss: 0.9432
Epoch 2/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 116us/step - accuracy: 0.7812 - loss: 0.6900 
Epoch 3/50


C:\Users\Acer\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


178/178 ━━━━━━━━━━━━━━━━━━━━ 38s 216ms/step - accuracy: 0.8330 - loss: 0.4551
Epoch 4/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 85us/step - accuracy: 0.8125 - loss: 0.7023  
Epoch 5/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 38s 213ms/step - accuracy: 0.8778 - loss: 0.3256
Epoch 6/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 71us/step - accuracy: 0.8750 - loss: 0.2499  
Epoch 7/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 43s 243ms/step - accuracy: 0.9135 - loss: 0.2409
Epoch 8/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 89us/step - accuracy: 0.9062 - loss: 0.2436  
Epoch 9/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 48s 267ms/step - accuracy: 0.9295 - loss: 0.1944
Epoch 10/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 106us/step - accuracy: 0.9375 - loss: 0.1331 
Epoch 11/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 51s 286ms/step - accuracy: 0.9402 - loss: 0.1454
Epoch 12/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 91us/step - accuracy: 0.9688 - loss: 0.1495  
Epoch 13/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 45s 253ms/step - accuracy: 0.9561 - loss: 0.1260
Epoch 14/50
178/178 ━━━━━━━━

In [8]:
import keras
model.save("finalized_model_kanker_otak.h5")
keras.saving.save_model(model, 'my_model.keras')

# Testing